# 밑바닥부터 트랜스포머 훈련하기

> **Note:** 이 장에서는 분산 인프라에서 대규모 언어 모델을 훈련하기 위한 대용량 데이터셋과 스크립트를 만듭니다. 따라서 이 노트북의 모든 단계가 코랩이나 캐글 같은 플랫폼에서 실행 가능한 것은 아닙니다. 중요 지점에서 단계를 축소하거나 분산 훈련 스크립트를 만들 때 참고용으로 이 노트북을 사용하세요.

> **Note:** 이 책의 다른 코드와 달리 이 장의 훈련 코드는 다중 GPU에서 스크립트로 실행하도록 만들어졌습니다. CodeParrot을 직접 훈련하려면  트랜스포머스 저장소([https://oreil.ly/ZyPPR](https://oreil.ly/ZyPPR))에 있는 스크립트를 사용하는 것이 좋습니다.

## 10.1 대규모 데이터셋 수집하기

- 깃허브 코파일럿
- 보일러플레이트 코드
- TabNine(https://tabnine.com)
- Kite(https://kite.com)
- CodeParrot

---

원하는 데이터가 모두 있을 때 무엇을 할 수 있는지 확인
- 사전 훈련 단계 자체를 살펴보고, 트랜스포머 모델을 밑바닥부터 훈련하는 법
- 이와 함께 여러 훈련의 측면 확인

> + 대용량 데이터 셋 수집과 처리
> + 자신만의 데이터셋을 위한 사용자 정의 토크나이저 만들기
> + 여러 GPU에서 대규모로 모델 훈련하기

---

파라미터가 수십억 개인 대규모 모델을 효과적으로 훈련하려면 분산 훈련을 위한 특별한 도구가 필요
- 분산훈련은 허깅페이스의 Trainer도 지원하지만,
- 허깅페이스 액셀러레이트(Accelerate)


**NOTE** CodeParrot을 직접 훈련하려면 허깅페이스 트랜스포머스 저장소(https://github.com/huggingface/transformers/tree/main/examples/research_projects/codeparrot)에 있는 스크립트 사용

### 10.1 대규모 말뭉치 구축의 어려움

사전 훈련용 대규모 말뭉치 구축에 관련된 일반적인 이슈와 도전 과제에 대해 인식 필요
- 대규모 데이터셋은 활동의 부산물로 생성된 데이터나 자동 혹은 반자동으로 양산된 데이터가 취합되었을 가능성이 큼
  + 회사에 저장된 문서, 사용자 활동 로그, 인터넷에서 수집한 데이터 등
- 대규모 데이텃이 고도의 자동화로 생성되었을 경우, 데이터셋의 내용과 생성 방법을 제어하기 어려워 편향되고 품질이 낮아지는 문제가 발생
- 최근 조사를 통해 BERT와 T5를 훈련하기 위해 각각 사용한 BookCorpus와 C4와 같은 유명한 대규모 데이터셋에서 밝혀진 사실
  + C4 말뭉치의 상당 부분을 사람이 아닌 기계가 번역함
  + C4의 불용어 필터링으로 아프리카계 미국인의 영어가 이질적으로 삭제되면서 관련 컨텐츠가 충분히 표현되지 못함
  + 대규모 텍스트 말뭉치는 성적인 컨텐츠와 노골적인 컨텐츠를 표현하거나 성과 젠더를 언급한 부분을 완전히 삭제함
  + BookCorpus는 저작권을 위반하는 내용이 많음(다른 대규모 데이터 셋도 마찬가지)
  + BookCorpus는 로맨스 소설 장르에 편향됨

---

데이터에 의해 왜곡된 모델의 개념을 설명하기 위해 GPT와 GPT-2에서 생성한 텍스트를 비교
- GPT는 대부분 BookCorpus에서 훈련하고
- GPT-2는 웹 페이지, 블로그, 레딧에 링크된 뉴스 기사에서 훈련함

크기가 비슷한 두 모델을 같은 프롬프트로 비교
- 두 모델의 차이점은 사전 훈련 데이터셋
- text-generation 파이프라인을 사용해 모델 출력을 조사


In [ ]:
!pip install datasets

In [ ]:
# text-generation 파이프 라인을 사용해 모델 출력을 조사

from transformers import pipeline, set_seed

generation_gpt = pipeline("text-generation", model="openai-gpt")
generation_gpt2 = pipeline("text-generation", model="gpt2")

In [3]:
# 각 모델의 파라미터 개수를 출력하는 간단한 함수 생성

def model_size(model):
    return sum(t.numel() for t in model.parameters())

print(f"GPT  크기: {model_size(generation_gpt.model)/1000**2:.1f}M parameters")
print(f"GPT2 크기: {model_size(generation_gpt2.model)/1000**2:.1f}M parameters")

GPT  크기: 116.5M parameters
GPT2 크기: 124.4M parameters


In [4]:
set_seed(1)

- 원본 GPT 모델은 가장 작은 GPT-2 모델과 크기가 거의 비슷함

- 두 모델에서 같은 프롬프트로 세 번의 자동 완성을 수행

In [5]:
def enum_pipeline_ouputs(pipe, prompt, num_return_sequences):
    out = pipe(prompt, num_return_sequences=num_return_sequences,
               clean_up_tokenization_spaces=True)
    return "\n".join(f"{i+1}." + s["generated_text"] for i, s in enumerate(out))

prompt = "\nWhen they came back"
print("GPT 자동 완성:\n" + enum_pipeline_ouputs(generation_gpt, prompt, 3))
print("")
print("GPT-2 자동 완성:\n" + enum_pipeline_ouputs(generation_gpt2, prompt, 3))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT 자동 완성:
1.
When they came back, she 'd want to know what he was going to do. 
 " it 'll teach your brother a lesson, " he said, and then he looked pointedly at my arm and the blood he 'd poured over my skin. "
2.
When they came back, a little boy with a black nose and brown eyes who had started hanging around the neighborhood on special occasions and who lived off of the property. he wore a football jersey with the letters " the black diamond " written on it.
3.
When they came back, and they were still there. " 
 jake nodded. 
 i shrugged and sat back down. " so then, they did come back, " i said, thinking. " but... " 
 " what's her name?

GPT-2 자동 완성:
1.
When they came back the body was still on fire, and there were a lot of people who had died. The smell of diesel had been getting worse. No one was doing well. The streets were deserted, with no water and no running water
2.
When they came back to the door, they came back with the fire extinguisher.

The man, who was carrying only 

- 두 모델에서 몇 개의 출력만 만들더라도, GPT가 생성한 문장에서 로맨스로 편향된 부분이 보임.

- 일반적으로 한 데이터셋에서 훈련한 모델은 언어 편향과 훈련 데이터에 있는 인구나 이벤트의 과대 또는 과소 표현을 반영함

- 모델 행동에 나타난 편향은 모델과 상호 작용하는 타깃 사용자와 관련해 중요하게 고려해야 함


### 10.1.2 사용자 정의 코드 데이터셋 만들기

10.1.1에서 대규모 텍스트 말뭉치를 만들 때 마주하게 될 어려움을 짧게 언급함
- 이를 유념하면서 자신만의 데이터셋 만들어 보기

사용자 정의 코드 데이터 셋을 마들기 위해 깃허브를 사용(2가지 방법 존재)
1. 9장에서 허깅페이스 트랜스포머스 저장소의 깃허브 이슈를 다운로드할 때 본 깃허브 REST API(https://docs.github.com/en/rest/using-the-rest-api/getting-started-with-the-rest-api?apiVersion=2022-11-28) 사용
2. **구글 빅쿼리(Big Query)** 같은 공개 데이터셋(https://console.cloud.google.com/marketplace/product/github/github-repos?pli=1) 사용


*아래 예제에서는 구글 빅쿼리 공개 데이터셋 활용*


#### 구글 빅쿼리로 데이터셋 만들기

1. 먼저 구글 빅쿼리 스냅샷(snapshot)에서 깃허브 공개 저장소에 있는 파이썬 파일을 모두 추출.
2. 동일한 재현과 향후 빅쿼리 무료 사용 정책이 변경되는 경우를 대비해 허깅페이스 허브에도 데이터셋을 공유
3. 파일을 추출하는 과정은 TransCoder(https://github.com/facebookresearch/TransCoder) 구현을 참조

> 1. 구글 클라우드 계정 생성
> 2. 계정 아래 구글 빅쿼리 프로젝트 생성
> 3. 이 프로젝트에서 데이터 셋 생성
> 4. 이 프로젝트에서 SQL 요청 결과를 저장할 테이블 생성
> 5. github_repos에서 다음 SQL 쿼리를 준비하고 실행

```SQL
SELECT
 f.repo_name,
 f.ref,
 f.path,
 c.copies,
 c.content
FROM `bigquery-public-data.github_repos.files` as f
  JOIN `bigquery-public-data.github_repos.contents` as c on f.id = c.id
WHERE
  NOT c.binary
  AND f.path like '%.py'
```

이 명령은 약 2.6TB 데이터를 처리해 2,680만개 파일을 추출함
- 결과 데이터셋은 약 50GB로 압축된 JSON 파일로 구성됨
- 각 파일에는 파이썬 파일의 소스 코드가 있음
  - 빈 파일과 유요한 정보가 적은 __init__.py 같은 작은 파일 데이터를 필터링함
  - 또 1MB보다 큰 파일도 필터링함
  - 나중에 라이선스에 따라 훈련 데이터를 필터링할 수 있도록, 파일의 라이센스도 모두 다운로드


---

결과 테이블을 로컬 컴퓨터로 다운로드

1. 결과를 구글 클라우드로 내보내기

  1.1 구글 클라우드 스토리지(GCS)에 버킷과 폴더를 생성
  1.2 `[내보내기]` > `[GCS로 내보내기]`를 선택하고 JSON 형식과 gzip 압축으로 테이블을 이 버킷으로 내보냄

2. 이 버킷을 컴퓨터로 다운로드하기 위해 gsutil 라이브러리 사용

  2.1 `pip install gsutil` 로 gsutil을 설치
  2.2 `gsutil config`를 실행해 gsutil에 구글 계정을 설정
  2.3 버킷을 컴퓨터로 복사

---

다음 명령으로 허깅페이스 허브에서 이 데이터셋을 바로 다운로드하는 방법도 있음

- `git clone https://huggingface.co/datasets/transformersbook/codeparrot`
- 클론 후 `git lfs pull`로 실제 데이터를 다운로드

##### 사이드바: 잡음 필터링 여부

- 사용 목적과 전체 시스템 통합 관점에 따라 잡음이 많거나 적은 데이터를 선택하고, 사전 필터링과 사후 필터링 연산을 추가함
- 데이터셋 정제의 고려사항
  + 프로그래밍 언어 간 균형 있는 비율을 유지할지
  + 품질이 낮은 데이터를 필터링할지
  + 중복된 코드 샘플을 삭제할지
  + 저작권 정보를 고려할지
  + 문서와 주석, docstring에 사용된 언어를 조사할지
  + 패스워드나 키 같은 개인 식별 정보를 삭제할 지 등

### 10.1.3 대용량 데이터셋 다루기

허깅페이스 데이터셋은 메모리와 하드 드라이브 공간의 제약을 해결할 수 있도록 메모리 매핑과 스트리밍 기능을 제공

#### 메모리 매핑

메모리 제약을 극복하기 위해 허깅페이스 데이터셋은 제로-카피(zero-copy)와 제로-오버헤드(zero-overhead) 메모리 매핑을 위한 메커니즘을 사용하며 기본적으로 활성화됨
- 각 데이터셋은 메모리 내용이 바로 반영되는 하나의 파일로 디스크에 캐싱됨
- 데이터셋을 메모리로 로딩하지 않고, 허깅페이스 데이터셋은 이 파일에서 읽기 전용 포인터(pointer)를 열어 메모리 대신 이를 사용함
  + 근본적으로 하드 드라이브를 메모리의 확장으로 사용



> **Note:** 다음 코드 블록은 빅쿼리 데이터셋을 `codeparrot` 폴더에 다운로드했다고 가정합니다. 압축 파일을 풀면 ~180GB 디스크 공간이 필요하기 때문에 이 단계를 건너 뛰는 것이 좋습니다. 이 코드는 예시 목적으로 쓴 것입니다. 대신 디스크 공간을 많이 사용하지 않는 스트리밍 데이터셋을 사용할 수 있습니다.

In [6]:
!mkdir codeparrot

여기서는 로컬 codeparrot 저장소에 저장된 50GB의 압축된 JSON 파일을 직접 로드함
- 이 때 180GB의 디스크 공간이 필요
- 램은 거의 사용하지 않음
- 데이터셋의 다운로드 설정에 delete_extracted=True를 지정하면 더 이상 필요하지 않은 모든 파일을 즉시 삭제함


In [27]:
from datasets import load_dataset, DownloadConfig


In [ ]:
'''
from datasets import load_dataset, DownloadConfig

download_config = DownloadConfig(delete_extracted=True)
dataset = load_dataset("./codeparrot", split="train",
                       download_config=download_config)
'''

허깅페이스 데이터셋은 내부적으로 압축된 JSON 파일을 최적화된 캐시 파일 하나에 로드해서 내용을 모두 추출하고 읽어 들임


In [ ]:
'''
import psutil, os

print(f"Number of python files code in dataset : {len(dataset)}")
ds_size = sum(os.stat(f["filename"]).st_size for f in dataset.cache_files)
# os.stat.st_size는 바이트 단위이므로 GB로 바꿉니다
print(f"Dataset size (cache file) : {ds_size / 2**30:.2f} GB")
# Process.memory_info는 바이트 단위이므로 MB로 바꿉니다
print(f"RAM used: {psutil.Process(os.getpid()).memory_info().rss >> 20} MB")
'''

데이터 셋이 일반적인 램 메모리 용량보다 훨씬 더 큼
- 이로 인해 훈련할 때 I/O 바운드 발생 가능
- 실제로 NLP 데이터를 로드하는 일은 모델 처리 계산에 비해 매우 가볍기 때문에 문제가 되는 경우가 극히 드뭄

#### 스트리밍

(1TB 또는 그 이상이 되는) 일부 데이터셋은 표준적인 하드 드라이브가 있더라도 다루기 어려움
- 이 경우 사용하는 서버의 용량을 높이는 대신 데이터셋을 스트리밍할 수 있음
- JSONL(JSON Lines), CSV, (원시 포맷 또는 zip, gzip, zstandard 압축된) 텍스트 같이 한 줄 씩 읽는 여러 종류의 압축 또는 비압축 파일 포맷을 허깅페이스 데이터셋으로 처리할 수 있음


In [ ]:
'''

# 캐시 파일을 만들지 않고 압축된 JSON 파일을 바로 데이터셋으로 로드

streamed_dataset = load_dataset('./codeparrot', split="train", streaming=True)

'''

데이터셋은 이제 IterableDataset 객체임
- 인덱스를 사용해 랜덤하게 원소에 접근할 수 없음
- 대신 next(iter(streamed_dataset)) 같은 방식으로 순서대로 읽어야 함

In [ ]:
'''
iterator = iter(streamed_dataset)

print(dataset[0] == next(iterator))
print(dataset[1] == next(iterator))
'''

스트리밍 데이터셋을 사용하면 데이터셋을 로드할 때 하드 드라이브에 캐시 파일이 생성되지 않고 많은 양의 메모리가 필요하지 않다는 장점이 있음
- 새로운 샘플 배치가 필요할 때 원시 파일을 즉시 추출하고 읽어 들여 해당 배치만 메모리에 로드함

이와 함께 로컬 데이터 셋이 아니라 허브에 있는 데이터 셋을 지정할 수 있음
- 그러면 원시 파일을 로컬에 다운로드하지 않고 샘플을 직접 다운로드함

In [ ]:
'''
remote_dataset = load_dataset('transformersbook/codeparrot', split="train",
                              streaming=True)
'''

- 이 데이터셋은 이전과 완전히 똑같이 동작함
- 하지만 내부적으로 샘플을 동적으로 다운로드 함
- 이렇게 설정하면 작은 서버에서도 아주 큰 대용량 데이터셋을 사용할 수 있음


### 10.1.4 허깅 페이스 허브에 데이터셋 추가하기

데이터 셋을 허깅페이스 허브에 업로드 하면 다음 작업이 가능함
- 훈련 서버에서 쉽게 다운로드 함
- 스트리밍 데이터셋을 허브 데이터셋과 함께 사용
- 커뮤니티와 공유

```
!huggingface-cli login
!huggingface-cli repo create --type dataset --organization transformersbook \ codeparrot-train
!huggingface-cli repo create --type dataset --organization transformersbook \ codeparrot-valid
```

여기서 저장소 타입을 데이터셋으로 지정하고 이 저장소가 속할 조직을 지정함
- 개인 계정에서 이 명령을 실행한다면 --organization 옵션을 빼도 좋음

비어 있는 두 저장소를 로컬 컴퓨터에 클론하고, JSON 파일을 각 저장소에 복사하고, 변경 사항을 허브에 푸쉬함

```
!git clone https://huggingface.co/datasets/transformersbook/codeparrot-train
!git clone https://huggingface.co/datasets/transformersbook/codeparrot-train
```
깃허브 파일을 제외한 모든 파일을 훈련 세트로 복사

```
!cd codeparrot-train
!cp ../codeparrot/*.json.gz .
!rm ./file-00000000183.json.gz
```

파일을 커밋하고 허브에 푸쉬함 (검증 세트에도 반복)
```
!git add .
!git commit -m "Adding dataset files"
!git push
```

두 데이터셋 분할과 전체 데이터셋이 아래 주소의 허깅페이스 허브에 준비됨

- https://huggingface.co/datasets/transformersbook/codeparrot

- https://huggingface.co/datasets/transformersbook/codeparrot-train

- https://huggingface.co/datasets/transformersbook/codeparrot-valid

## 10.2 토크나이저 구축하기

1. 사전 훈련 모델 사용 시

- 선택한 전처리 방식 사용 필요

   + 그렇지 않으면 분포를 벗어난 패턴이나 알 수 없는 토큰이 모델에 입력

2. 새로운 모델 훈련 시 새로운 토크나이저 구축 필요

- 새로운 모델 훈련 시 기존 토크나이저 사용할 때 발생 문제 유형

  + T5 토크나이저는 불용어 필터링이 광범위하게 적용됨. 모르는 단어가 많을 수 있음

  + CamemBERT 토크나이저도 대규모 텍스트 말뭉치에서 훈련되었지만, 이 데이터셋은 프랑스어 텍스트로만 구성됨. 따라서 평범한 영어 단어를 인식하기 어려울 수 있음

In [7]:
from transformers import AutoTokenizer

def tok_list(tokenizer, string):
    input_ids = tokenizer(string, add_special_tokens=False)["input_ids"]
    return [tokenizer.decode(tok) for tok in input_ids]

tokenizer_T5 = AutoTokenizer.from_pretrained("t5-base")
tokenizer_camembert = AutoTokenizer.from_pretrained("camembert-base")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

In [8]:
print(f'"sex"에 대한 T5 토큰: {tok_list(tokenizer_T5,"sex")}')
print(f'"being"에 대한 CamemBERT 토큰: {tok_list(tokenizer_camembert,"being")}')

"sex"에 대한 T5 토큰: ['', 's', 'ex']
"being"에 대한 CamemBERT 토큰: ['be', 'ing']


많은 경우, 이렇게 짧고 평범한 단어를 부분으로 나누면 모델이 입력되는 시퀀스 길이가 늘어나서 비효율적이게 됨
- 따라서 토크나이저를 훈련하는데 사용한 데이터셋의 도메인과 전처리 방식을 이해하는 것이 중요
- 토크나이저와 모델은 데이터셋의 편향을 인코딩할 수 있는데, 이는 모델의 후속 행동에 영향을 미침
- 데이터셋에 맞는 최적의 토크나이저를 얻으려면 토크나이저를 직접 훈련해야 함

**NOTE** 모델 훈련은 일련의 가중치로 시작함. 그 다음 역전파를 사용해 목적 함수의 오류 신호로부터 모델의 손실을 최소화함. 결국 훈련이란 특정 목적하에서 작업을 수행할 최적의 모델 가중치 집합을 찾는 과정임
- 반면 토크나이저의 훈련은 역전파나 가중치와 무관함
- 토크나이저 훈련은 **텍스트 문자영을 정수 리스트로 매핑해서 모델에 주입할 최적의 매핑을 찾는 과정**
- 오늘날 토크나이저에서 최적의 문자열-정수 변환에는 단위(atomic) 문자열의 리스트로 구성된 어휘사전과 변환, 정규화, 잘라내기, 텍스트 문자열을 인덱스 리스트로 매핑하기 등을 위한 메서드가 사용됨. 그 다음 이 인덱스의 리스트가 신경망의 입력이 됨

### 10.2.1 토크나이저 모델

4장에서 보았듯이 토크나이저는 정규화 -> 사전 토큰화 -> 토크나이저 모델 -> 사후 처리 네 단계로 구성된 전처리 파이프 라인
- 데이터로 훈련하는 토크나이저 파이프라인 부분이 토크나이저 모델임
- BPE, WordPiece, 유니그램 같은 부분 단어 토큰화 알고리즘을 사용

**(BPE)** 기본 단위(단일 문자)의 리스트로 시작해서 가장 자주 함께 등장한 기본 단위를 합쳐 어휘사전에 추가하는 식으로 점진적으로 새 토큰을 만드는 과정을 거쳐 어휘사전을 만듬

**(유니그램)** 말뭉치에 있는 모든 단어와 가능성 있는 부분단어로 기본 어휘사전을 구성. 그 다음 어휘사전의 목표 크기에 도달할 때까지 점진적으로 유용성이 떨어지는 토큰을 삭제하거나 분할해 더 작은 어휘 사전을 얻음(**WordPiece**는 유니그램의 전신이며 공식 구현은 구글에서 오픈소스로 공개한 적이 없음)


### 10.2.2 토크나이저 성능 측정하기

토크나이저의 성능을 측정하는 방법

1. **부분단어 생산력(subword fertility)** 토큰화된 단어마다 생성되는 부분단어의 평균 개수를 계산

2. **연속 단어의 비율(proportion of continued words)** 말뭉치에서 적어도 두 개의 부분 토큰으로 분할된 토큰화된 단어의 비율

3. **커버리지 측정값(coverage metrics)** 토큰화된 말뭉치에서 알 수 없는 단어나 거의 사용되지 않는 토큰의 비율

결국 다양한 토큰화 방식의 성능은 일반적으로 후속 모델의 성능을 궁극적인 지표로 사용할 때 가장 잘 추정됨
- 예를 들어 초기 BPE 방식의 성능이 우수하다는 사실을 입증하기 위해 문자나 단어 기반 토큰화 대신 BPE 방식의 토크나이저와 어휘사전을 사용해 훈련한 모델이 기계 번역 작업의 성능을 향상시킴을 보임


### 10.2.3 파이썬 코드를 위한 토크나이저

파이썬 코드를 토큰화할 사용자 정의 토크나이저 생성
- 코드 토큰화에 자연어 처리 토크나이저를 사용하는 것이 최적은 아님

**NOTE** 파이썬은 파이썬 코드를 의미 있는 단위(코드 연산, 주석, 들여쓰기, 내어 쓰기 등)로 분할하는 tokenize  모듈을 내장하고 있음. 이 모듈을 사용할 때 문제점은 이 토크나이저가 파이썬 기반이라 보통 느리고 파이썬의 GIL(Global Interperter Lock) 때문에 성능이 제한됨
- 반면 허깅페이스 트랜스포머스 라이브러리에 있는 대부분의 토크나이저는 허깅페이스 토크나이저 라이브러리에서 제공하며 러스트(RUST)로 작성됨.
- 러스트 토크나이저는 수십 배 이상 빠른 훈련과 실행이 가능함


In [29]:
# gpt2의 오토 토크나이저를 활용

from transformers import AutoTokenizer

python_code = r"""def say_hello():
    print("Hello, World!")
# Print it
say_hello()
"""
tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!"', ')', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


출력이 이상한 부분을 확인할 수 있음
- 토크나이저에 적용된 정규화 방법 확인

In [10]:
print(tokenizer.backend_tokenizer.normalizer)

None


GPT-2 토크나이저는 정규화를 사용하지 않음
- 어떤 정규화 단계도 거치지 않고 원시 유니코드 입력을 바로 사용함

In [11]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(python_code))

[('def', (0, 3)), ('Ġsay', (3, 7)), ('_', (7, 8)), ('hello', (8, 13)), ('():', (13, 16)), ('ĊĠĠĠ', (16, 20)), ('Ġprint', (20, 26)), ('("', (26, 28)), ('Hello', (28, 33)), (',', (33, 34)), ('ĠWorld', (34, 40)), ('!")', (40, 43)), ('Ċ', (43, 44)), ('#', (44, 45)), ('ĠPrint', (45, 51)), ('Ġit', (51, 54)), ('Ċ', (54, 55)), ('say', (55, 58)), ('_', (58, 59)), ('hello', (59, 64)), ('()', (64, 66)), ('Ċ', (66, 67))]


허깅페이스 토크나이저 동작 방식

1. **오프셋 트래킹(offset tarcking)** 입력 문자열의 위치를 기억하는 기능

2. **바이트 단위 동작** 바이트 알파벳 사용
  + 유니코드 문자를 바이트 시퀀스로 변환 가능
  

In [12]:
a, e = u"a", u"€"
byte = ord(a.encode("utf-8"))
print(f'`{a}`는 단일 바이트 `{a.encode("utf-8")}`로 인코딩됩니다: {byte}')
byte = [ord(chr(i)) for i in e.encode("utf-8")]
print(f'`{e}`는 세 바이트 `{e.encode("utf-8")}`로 인코딩됩니다: {byte}')

`a`는 단일 바이트 `b'a'`로 인코딩됩니다: 97
`€`는 세 바이트 `b'\xe2\x82\xac'`로 인코딩됩니다: [226, 130, 172]


**BPE** 알고리즘 수행 방식 적용
- 가장 자주 등장한 바이트 조합 256개 단어 어휘를 확장해서 어휘 사전을 중간 규모로 만듬

**BPE 알고리즘 사용 시 문제점**
- 입력을 바이트가 아니라 정제된 유니코드 문자열을 다루도록 설계되어 공백이나 제어 문자가 아닌 일반적인 ASCII 문자를 받음
- 하지만 처음 256개 바이트 유니코드 문자에는 제어 문자(줄바꿈, 탭, 그 외 화면에 출력되지 않는 문자 등)가 많이 있음
- GPT-2 토크나이저는 256개 입력 바이트를 표준 BPE 알고리즘이 쉽게 처리하는 유니코드 문자열로 매핑함



In [30]:
# GPT-2 토크나이저의 매핑 결과 확인

from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

print(f'기본 어휘 사전 크기: {len(base_vocab)}')
print(f'첫 번째 원소: `{base_vocab[0]}`, last element: `{base_vocab[-1]}`')

기본 어휘 사전 크기: 256
첫 번째 원소: `!`, last element: `Ń`


**[표 10-1]**에 바이트 값과 매핑된 유니코드 문자 몇 가지를 열거

In [31]:
# BPE 문자 매핑의 예
import pandas as pd
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

examples = [
    ['Regular characters', '`a` and `?`', f'{ord("a")} and {ord("?")}' , f'`{byte_to_unicode_map[ord("a")]}` and `{byte_to_unicode_map[ord("?")]}`'],
    ['Nonprintable control character (carriage return)', '`U+000D`', f'13', f'`{byte_to_unicode_map[13]}`'],
    ['A space', '` `', f'{ord(" ")}', f'`{byte_to_unicode_map[ord(" ")]}`'],
    ['A nonbreakable space', '`\\xa0`', '160', f'`{byte_to_unicode_map[ord(chr(160))]}`'],
    ['A newline character', '`\\n`', '10', f'`{byte_to_unicode_map[ord(chr(10))]}`'],
]

pd.DataFrame(examples, columns = ['Description', 'Character', 'Bytes', 'Mapped bytes'])

,Description,Character,Bytes,Mapped bytes
0,Regular characters,`a` and `?`,97 and 63,`a` and `?`
1,Nonprintable control character (carriage return),`U+000D`,13,`č`
2,A space,` `,32,`Ġ`
3,A nonbreakable space,`\xa0`,160,`ł`
4,A newline character,`\n`,10,`Ċ`


- 줄바꿈을 NEWLINE 문자열로 매핑하는 식으로 조금 더 명시적인 변환을 할 수 있지만, BPE 알고리즘은 일반적으로 문자를 다루도록 고안됨
- 이런 이유에서 각 바이트 문자에 하나의 유니코드 문자를 매핑해야 기존의 BPE 알고리즘이 처리하기 쉬움

In [15]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(python_code))

[('def', (0, 3)), ('Ġsay', (3, 7)), ('_', (7, 8)), ('hello', (8, 13)), ('():', (13, 16)), ('ĊĠĠĠ', (16, 20)), ('Ġprint', (20, 26)), ('("', (26, 28)), ('Hello', (28, 33)), (',', (33, 34)), ('ĠWorld', (34, 40)), ('!")', (40, 43)), ('Ċ', (43, 44)), ('#', (44, 45)), ('ĠPrint', (45, 51)), ('Ġit', (51, 54)), ('Ċ', (54, 55)), ('say', (55, 58)), ('_', (58, 59)), ('hello', (59, 64)), ('()', (64, 66)), ('Ċ', (66, 67))]


GPT-2 토크나이저의 어휘사전은 50,257개 단어로 구성됨
- 기본 어휘사전은 256개 바이트 값에 해당함
- 50,000개 추가 토큰은 가장 자주 함께 등장한 토큰을 반복적으로 합쳐 생성
- 문서 경계를 나타내는 특별한 문자가 어휘사전에 추가됨


In [16]:
print(f"어휘 사전의 크기: {len(tokenizer)}")

어휘 사전의 크기: 50257


위 구성 내용을 어휘 사전 크기 속성을 통해 쉽게 확인함

- 샘플 입력 코드에 전체 파이프라인을 실행하면 다음과 같은 결과를 얻음

In [17]:
print(tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!"', ')', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


이처럼 BPE 토크나이저는 대부분 단어를 유지하지만 들여쓰기에 있는 여러 공백을 몇 개의 연속된 공백으로 나눔
- 코드에서 훈련되지 않았기 때문
- 대부분 텍스트에서 연속된 공백을 드문 경우

따라서 BPE 모델은 들여쓰기를 위해 어휘사전에 특수 토큰을 포함하지 않음
- 바로 이 경우가 토크나이저 모델이 주어진 데이터셋의 도메인에 잘 맞지 않는 한 예

앞에서 논의하였듯 타깃 말뭉치에서 토크나이저를 재훈련하는 것이 해결책

### 10.2.4 토크나이저 훈련하기

파이썬 코드에 잘 맞는 어휘사전을 얻기 위해 이 예제의 말뭉치에서 바이트 수준 BPE 토크나이저를 재훈련을 실시
- 허깅페이스 트랜스포머스가 제공하는 토크나이저는 간단하게 재훈련되는데, 다음 내용이 필요함
  + 목표 어휘사전의 크기를 지정함
  + 토크나이저 모델을 훈련하기 위해 입력 문자열을 공급할 반복자(iterator)를 준비
  + train_new_from_iterator() 메서드를 호출

- 딥러닝 모델은 훈련 말뭉치에서 특정 세부 내용을 많이 기억하도록 훈련하지만, 토크나이저는 주요 통계값을 추출하도록 훈련함. 간단히 말해 토크나이저는 말뭉치에서 가장 자주 등장한 문자 조합을 알아내는 훈련을 함

  +  따라서 토크나이저를 반드시 대규모 말뭉치에서 훈련할 필요가 없음
  + 훈련 말뭉치는 해당 도메인을 대표하고 토크나이저가 통계적으로 의미 있는 값을 추출할 정도로 크면 됨
  + 어휘 사전의 크기와 말뭉치에 있는 텍스트에 따라 토크나이저가 예상치 못한 단어를 저장할 수 있음

예를 들어 GPT-2 토크나이저의 어휘 사전에서 가장 긴 단어를 확인하면 이런 현상을 볼 수 있음

In [24]:
from transformers import GPT2Tokenizer

# GPT-2 토크나이저 초기화
tokenizer2 = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer2

# 어휘 사전을 가져옴
vocab = tokenizer2.get_vocab()

# 가장 긴 토큰 찾기
longest_token = max(vocab.keys(), key=len)

print("가장 긴 토큰:", longest_token)
print("길이:", len(longest_token))

가장 긴 토큰: ÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤÃĥÃĤ
길이: 128


첫 토큰 <|endoftext|>는 텍스트 시퀀스의 끝을 지정할 때 사용하는 특수 토큰으로 BPE 어휘 사전이 구축된 후 추가됨
- 모델은 각 토큰에 관련된 단어 임베딩을 학습해야 하는데 임베딩 행렬에 잡음 단어가 많이 포함되어 있다면 좋지 않음
- 또한 세상의 시공간적 지식을 어휘사전에 있는 벡터와 함께 별개의 토큰으로 부여해 저수준에서 이를 임베딩할 수 있음
- BPE 토크나이저가 이런 토큰을 생성한다면 목표 어휘사전이 너무 크거나 말뭉치에 특수 토큰이 포함되었다는 신호가 됨

In [25]:

# 가장 높은 고유 ID를 가진 토큰 찾기
rarest_token = max(vocab, key=vocab.get)

print("가장 높은 고유 ID를 가진 토큰:", rarest_token)
print("고유 ID:", vocab[rarest_token])

가장 높은 고유 ID를 가진 토큰: <|endoftext|>
고유 ID: 50256


예제의 말뭉치에서 새 토크나이저를 훈련하고 학습된 어휘사전을 살펴보겠음
- 데이터셋의 통계를 대표할 말뭉치가 필요하니 말뭉치에서 약 100,000개 문서를 선택

In [36]:
from tqdm.auto import tqdm

length = 10000
dataset_name = 'transformersbook/codeparrot-train'
dataset = load_dataset(dataset_name, split="train", streaming=True)
iter_dataset = iter(dataset)

def batch_iterator(batch_size=10):
    for _ in tqdm(range(0, length, batch_size)):
        yield [next(iter_dataset)['content'] for _ in range(batch_size)]

new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(),
                                                  vocab_size=1250,
                                                  initial_alphabet=base_vocab)

Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

이 어휘사전이 작업과 얼마나 관련되었는지 알아보기 위해 BPE 알고리즘이 만든 첫 단어와 마지막 단어를 조사
- 256 바이트는 건너 뛰고 그 다음에 추가된 첫 토큰부터 살펴봄

In [37]:
# tokens = sorted(new_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)
# print([f'{tokenizer.convert_tokens_to_string(t)}' for t, _ in tokens[257:280]]);

# GPT2Tokenizer의 경우, 각 토큰은 그 자체로 문자열이므로,
# 리스트 컴프리헨션 내에서 단일 토큰 t를 직접 사용할 수 있음

tokens = sorted(new_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)
print([t for t, _ in tokens[257:280]]);


['ĠĠ', 'ĠĠĠĠ', 'ĠĠĠ', 'ĠĠĠĠĠĠĠĠ', 'se', 'in', 're', 'ĠĠĠĠĠĠĠ', 'on', 'te', 'ĊĠĠĠĠĠĠĠ', 'ĊĠĠĠĠĠĠĠĠ', 'or', 'st', 'de', 'ĊĠĠĠ', 'th', 'Ġ=', 'le', 'lf', 'self', 'me', 'al']


여러 수준의 들여쓰기와 공백 토큰, self, or, in 같은 짧은 파이썬 예약어가 보임
- 이는 BPE 알고리즘이 의도대로 동작한다는 신호

In [40]:
#print([f'{new_tokenizer.convert_tokens_to_string(t)}' for t, _ in tokens[-12:]]);
print([t for t, _ in tokens[-12:]]);

['mber', 'dc', 'length', 'ite', 'cs', 'tern', 'sible', 'sys', 'gger', 'ump', 'sions', 'header']


header와 같이 주석에서 나오는 단어들도 확인됨

In [41]:
print(new_tokenizer(python_code).tokens())

['def', 'Ġs', 'a', 'y', '_', 'h', 'el', 'lo', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'H', 'el', 'lo', ',', 'ĠW', 'or', 'l', 'd', '!', '")', 'Ċ', '#', 'ĠP', 'rint', 'Ġit', 'Ċ', 's', 'a', 'y', '_', 'h', 'el', 'lo', '()', 'Ċ']


In [42]:
import keyword

print(f'파이썬 전체 예약어 개수: {len(keyword.kwlist)}')
for keyw in keyword.kwlist:
    if keyw not in new_tokenizer.vocab:
        print(f'예약어 `{keyw}`는 어휘 사전에 없습니다.')

파이썬 전체 예약어 개수: 35
예약어 `async`는 어휘 사전에 없습니다.
예약어 `await`는 어휘 사전에 없습니다.
예약어 `break`는 어휘 사전에 없습니다.
예약어 `continue`는 어휘 사전에 없습니다.
예약어 `del`는 어휘 사전에 없습니다.
예약어 `elif`는 어휘 사전에 없습니다.
예약어 `else`는 어휘 사전에 없습니다.
예약어 `except`는 어휘 사전에 없습니다.
예약어 `finally`는 어휘 사전에 없습니다.
예약어 `global`는 어휘 사전에 없습니다.
예약어 `lambda`는 어휘 사전에 없습니다.
예약어 `nonlocal`는 어휘 사전에 없습니다.
예약어 `pass`는 어휘 사전에 없습니다.
예약어 `raise`는 어휘 사전에 없습니다.
예약어 `while`는 어휘 사전에 없습니다.
예약어 `yield`는 어휘 사전에 없습니다.


파이썬의 예약어가 모두 어휘사전에 있는 지 확인

- new_tokenizer의 어휘 사전에는 elif와 같은 파이썬 예약어가 없다고 나타남

어휘 사전의 개수를 늘리고 학습량을 늘림

1.   어휘사전을 32,768개 단어로 구성(8의 배수가 일부 GPU/TPU 계산에 더 효율적)

그 이유에 대해 chatGPT는 아래와 같이 대답해 줌
---
GPU나 TPU와 같은 병렬 컴퓨팅 하드웨어에서 하이퍼파라미터를 8의 배수로 설정하는 것이 계산에 더 효율적인 이유는 이러한 하드웨어 아키텍처가 데이터를 병렬로 처리하는 방식과 관련이 있습니다. 특히, 이는 배치 크기(batch size), 모델의 차원수 같은 하이퍼파라미터에 적용됩니다. 몇 가지 주요 이유는 다음과 같습니다:

병렬 처리와 메모리 접근
병렬 처리 최적화: GPU와 TPU는 병렬 처리를 위해 설계되었습니다. 이러한 장치들은 동시에 많은 연산을 수행할 수 있으며, 특히 데이터가 정해진 패턴을 따를 때 가장 효율적입니다. 8의 배수로 설정된 하이퍼파라미터는 이러한 장치들이 내부적으로 가지고 있는 병렬 처리 유닛(예: CUDA 코어, TPU 코어)에 데이터를 더 균일하게 분배하게 해주어, 각 유닛의 처리 능력을 최대로 활용할 수 있게 합니다.

메모리 접근 효율성: 하이퍼파라미터를 8의 배수로 설정하면 메모리 접근 패턴이 더 효율적이 됩니다. 대부분의 GPU와 TPU는 메모리에서 데이터를 읽고 쓸 때 특정 크기의 블록 단위로 작업을 수행합니다. 8의 배수는 이러한 블록 단위와 잘 맞아떨어지므로, 메모리 대역폭을 보다 효율적으로 활용할 수 있습니다.

연산 최적화
텐서 코어와 매트릭스 연산: 최신 GPU와 TPU는 행렬 연산을 가속화하는 특수 하드웨어 유닛(예: NVIDIA의 텐서 코어)을 포함하고 있습니다. 이러한 유닛들은 특정 크기의 행렬(보통 8x8, 16x16 등)을 더 효율적으로 처리할 수 있으며, 하이퍼파라미터가 이러한 크기와 맞아떨어질 때 연산을 더 빠르게 수행할 수 있습니다.

메모리 패딩과 정렬: 8의 배수로 하이퍼파라미터를 설정하면 메모리 내에서 데이터가 더 잘 정렬됩니다. 이는 필요한 데이터를 연속적인 메모리 블록에서 더 쉽게 접근할 수 있게 하여, 캐시 미스(cache miss)를 줄이고 전체적인 성능을 향상시킵니다.

실제 효과
이러한 최적화는 특히 대규모 모델과 데이터셋을 사용하는 딥러닝 작업에서 중요합니다. 계산 시간과 에너지 소비를 줄이며, 전반적인 학습 효율을 개선할 수 있습니다. 그러나, 실제 효과는 사용하는 하드웨어, 모델의 구조, 그리고 특정 작업의 성격에 따라 다를 수 있습니다.



In [43]:
length = 200000
new_tokenizer_larger = tokenizer.train_new_from_iterator(batch_iterator(),
    vocab_size=32768, initial_alphabet=base_vocab)

  0%|          | 0/20000 [00:00<?, ?it/s]

마지막 토큰 확인

In [45]:
tokens = sorted(new_tokenizer_larger.vocab.items(), key=lambda x: x[1], reverse=False)
print([t for t, _ in tokens[-12:]]);

['stacktrace', 'shortname', 'Ġstripping', 'masters', 'Ġcomplexity', 'DISC', 'Ġindividuals', 'SYSZ', 'lligence', 'pyspark', 'HIGHEST', 'RichTextBlock']


새 토크나이저로 샘플 코드를 토큰화 함

In [46]:
print(new_tokenizer_larger(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


여기서도 편리하게 들여쓰기가 어휘사전에 유지됨
- Hello, World, say와 같은 평범한 영어 단어도 하나의 토큰으로 포함됨

In [47]:
for keyw in keyword.kwlist:
    if keyw not in new_tokenizer_larger.vocab:
        print(f'예약어 `{keyw}`는 어휘 사전에 없습니다.')

예약어 `nonlocal`는 어휘 사전에 없습니다.


- 어휘 사전에 없는 예약어가 획기적으로 줄어든 것을 확인 가능

> **NOTE** 토큰화된 코드 샘플의 시퀀스 길이를 비교해서 새 토크나이저가 기본 GPT-2 토크나이저보다 거의 두 배 더 효율적임을 쉽게 확인함
- 새 토크나이저는 기존 토크나이저가 텍스트를 인코딩할 때 사용한 토큰의 약 절반만 사용함
- 이로 인해 아무런 비용을 들이지 않고 실제적인 모델의 문맥 크기가 두 배로 늘어남
- 문맥 윈도 크기 1,024에서 새로운 토크나이저로 모델을 훈련하는 것은 문맥 윈도 크기 2,048에서 기존 토크나이저로 모델을 훈련하는 것과 동일함
- 하지만 훨씬 더 빠르고 메모리 효율성도 더 높은 이점이 있음


### 10.2.5 허브에 사용자 정의 토크나이저 저장하기

훈련한 토크나이저를 저장함

```
# 로그인 과정을 거친 후 아래 코드 실행

from huggingface_hub import notebook_login

notebook_login()
```


In [49]:
from huggingface_hub import notebook_login

notebook_login()

In [50]:
model_ckpt = "codeparrot"
# org = "transformersbook"
new_tokenizer_larger.push_to_hub(model_ckpt) # , organization=org)

CommitInfo(commit_url='https://huggingface.co/youngbreadho/codeparrot/commit/b60ae14803b8c453590556d42fab64d00a654d1e', commit_message='Upload tokenizer', commit_description='', oid='b60ae14803b8c453590556d42fab64d00a654d1e', pr_url=None, pr_revision=None, pr_num=None)

어떤 조직 내에 푸시하고 싶지 않다면 organization 매개변수를 제외함
- 자신의 네임스페이스 안에 codeparrot란 이름의 저장소가 만들어지고 누구든지 아래 코드를 실행해 토크나이저를 로드할 수 있음

In [53]:
reloaded_tokenizer = AutoTokenizer.from_pretrained("codeparrot/" + model_ckpt)
print(reloaded_tokenizer(python_code).tokens())

tokenizer_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/497k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/277k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

['def', 'Ġsay', '_', 'hello', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


허브에서 로드한 토크나이저는 동일한 동작을 수행

- 나중에 재현할 수 있도록 작은 버전의 토크나이저도 저장

In [54]:
new_tokenizer.push_to_hub(model_ckpt+ "-small-vocabulary") #, organization=org)

CommitInfo(commit_url='https://huggingface.co/youngbreadho/codeparrot-small-vocabulary/commit/58b21aefe565dbaa6d14511811f0093cdd544022', commit_message='Upload tokenizer', commit_description='', oid='58b21aefe565dbaa6d14511811f0093cdd544022', pr_url=None, pr_revision=None, pr_num=None)

## 10.3 밑바닥부터 모델을 훈련하기

이 절에서는 작업에 가장 잘 맞는 아키텍처를 결정하고, 사전 훈련된 가중치 없는 새 모델을 초기화한 뒤, 사용자 정의 데이터 로딩 클래스를 만들고, 확장 가능한 훈련 루프를 생성
- 최종적으로 각각 파라미터가 1억 1,100만개인 소규모 GPT-2 모델과 15억개인 대규모 GPT-2 모델을 훈련

**TIP** 이 절에서는 분산 환경에서 모델을 훈련하기 위해 스크립트를 보통 길이보다 더 길게 구현함
따라서 코드 조각을 독립적으로 실행하지 말고 허깅페이스 트랜스포머스 저장소에 있는 스크립트(https://github.com/huggingface/transformers/tree/main/examples/research_projects/codeparrot)를 활용

### 10.3.1 사전 훈련 목표

아래 그림과 같은 코드 조각으로 구성된 대용량 코드 베이스를 사용해 여러가지 작업을 처리

<img alt="Code snippet" caption="An example of a Python function that could be found in our dataset" src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter10_code-snippet.png?raw=1" id="code-snippet"/>

**[그림 10-1]** 데이터셋에서 찾을 수 있는 파이썬 함수의 예


#### **A. 코잘 언어 모델링**

**코잘 언어 모델링(causal language modeling)** 레이블이 없는 데이터셋을 사용하는 자기 지도 훈련 목표 학습
- 코드 샘플 시작 부분을 모델에게 제공하고 코드의 나머지 부분을 생성해 완성하라고 요청하는 것
- 코드 자동 완성은 이와 직접적으로 관련된 후속 작업
- 이런 작업에는 일반적으로 아래 그림과 같이 GPT 계열의 디코더 전용 아키텍처가 가장 잘 맞음

<img alt="CLM pretraining" caption="In causal language modeling, the future tokens are masked and the model has to predict them; typically a decoder model such as GPT is used for such a task" src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter10_pretraining-clm.png?raw=1" id="pretraining-clm"/>

**[그림 10-2] 코잘 언어 모델링 구조도**
- 코잘 언어 모델링에서 모델은 마스킹된 미래 토큰을 예측함
- 이런 작업에는 일반적으로 GPT 같은 디코더 모델을 사용

#### **B. 마스크드 언어 모델링**

**마스크드 언어 모델링**(masked language modeling) 또는 **잡음 제거 목표**(denoising objective)
- 모델에게 잡음이 섞인 코드 샘플을 주고 깨끗한 원본 샘플을 재구성하라고 요청하는 방식
- 가령 랜덤한 단어나 마스킹된 단어로 코드 명령을 바꿈
- 잡음 제거와 직접적으로 관련된 후속 작업을 생각하기 어렵지만, 대체로 잡음 제거는 후속 작업을 위해 보편적인 표현을 학습하기 좋은 사전 훈련 작업
- 이전 장에서 사용한(BERT와 XLM-RoBERTa 같은) 많은 모델들이 이런 식으로 훈련함
  + 레이블링된 샘플이 제한적인 후속 작업의 경우 대규모 말뭉치에서 훈련된 마스크드 언어 모델을 사용해 모델을 미세 튜닝할 수 있음


<img alt="MLM pretraining" caption="In masked language modeling some of the input tokens are either masked or replaced, and the model's task is to predict the original tokens; this is the architecture underlying the encoder branch of transformer models" src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter10_pretraining-mlm.png?raw=1" id="pretraining-mlm"/>

**[그림 10-3] 마스크드 언어 모델링 구조도**
- 입력 토큰 중 일부가 마스킹되거나 바뀜
- 원본 토큰을 예측하는 것이 모델의 작업임
- 트랜스포머 모델 중 인코더의 기반이 되는 아키텍처

#### **C. 시퀀스-투-시퀀스 훈련**

정규식 같은 수동 규칙으로 주석이나 독스트링을 코드에서 분리하고 레이블링된 데이터셋으로 사용하도록 (코드, 주석) 쌍의 대규모 데이터 셋을 구축하는 작업

- 훈련은 한 카테고리(코드나 주석)를 모델의 입력으로 사용하고 다른 카테고리(주석이나 코드)를 레이블로 사용하는 지도 학습 목표가 됨
- 다양성을 갖춘 대규모의 정제된 데이터셋과 용량이 충분한 모델을 사용해 코드에 있는 코드에서 주석을 예측하거나 그 반대를 학습하는 모델을 훈련할 수 있음

이런 지도 학습 훈련 작업에 직접적으로 관련된 후속 작업은  문서를 생성하거나 문서로부터 코드를 생성하는 일이 됨
- 이런 설정에서 한 시퀀스는 다른 시퀀스로 변환되며 T5, BART, PEGASUS 같은 인코더-디코더 모델이 여기에 잘 맞음


<img alt="Seq2seq pretraining" caption="Using an encoder-decoder architecture for a sequence-to-sequence task where the inputs are split into comment/code pairs using heuristics: the model gets one element as input and needs to generate the other one" src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter10_pretraining-seq2seq.png?raw=1" id="pretraining-seq2seq"/>

**[그림 10-4] 시퀀스 투 시퀀스 모델 구조도**
- 인코더-디코더 아키텍처 사용
- 입력은 경험적인 규칙을 사용해 주석/코드 쌍으로 분할되고
- 모델은 주석과 코드 중 하나를 입력으로 받아 다른 하나를 생성


### 10.3.2 모델 초기화

> **NOTE**: 다음 코드 블록에서 대용량 GPT-2 체크포인트를 메모리에 로드합니다. 코랩이나 캐글 같은 플랫폼에서는 램이나 GPU 메모리가 부족하기 때문에 인스턴스가 종료될 수 있습니다. 이런 경우 `config = AutoConfig.from_pretrained("gpt2", vocab_size=len(tokenizer))`로 설정을 바꾸어 작은 체크포인트를 사용하면 이 예제를 실행할 수 있습니다.

- "gpt2-xl" -> "gpt2"

In [57]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

org = "codeparrot"

tokenizer = AutoTokenizer.from_pretrained(org + "/" + model_ckpt)
config = AutoConfig.from_pretrained("gpt2", vocab_size=len(tokenizer))
model = AutoModelForCausalLM.from_config(config)

In [58]:
print(f'GPT-2 크기: {model_size(model)/1000**2:.1f}M parameters')

GPT-2 크기: 111.0M parameters


In [59]:
model.save_pretrained("models/" + model_ckpt, push_to_hub=True,
                      organization=org)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:703: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


model.safetensors:   0%|          | 0.00/444M [00:00<?, ?B/s]

In [60]:
model_small = model

print(f'GPT-2 크기: {model_size(model_small)/1000**2:.1f}M parameters')

GPT-2 크기: 111.0M parameters


In [61]:
model_small.save_pretrained("models/" + model_ckpt + "-small", push_to_hub=True,
                            organization=org)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:703: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


model.safetensors:   0%|          | 0.00/444M [00:00<?, ?B/s]

### 10.3.3 데이터로더 구축하기



여러 샘플을 토큰화한 다음 특수한 EOS 토큰으로 이를 연결해 매우 긴 시퀀스를 만듬

아래 그림처럼 이 시퀀스를 동일한 크기의 청크(chunk)로 나눔
- 이 방법을 사용하면 마지막 데이터에서 손실되는 부분이 미미함

<img alt="Preprocessing for CLM" caption="Preparing sequences of varying length for causal language modeling by concatenating several tokenized examples with an EOS token  before chunking them" src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter10_preprocessing-clm.png?raw=1" id="preprocessing-clm"/>

**[그림 10-5] 코잘 언어 모델링을 위해 가변 길이 시쿼스 준비하기**

EOS 토큰으로 토큰화된 샘플을 여러개 연결한 다음 청크로 나눔

예를 들어 입력 문자열의 문자 개수를 다음과 같이 정의해서 토큰화된 샘플에 약 100개의 완전한 시퀀스가 있게 만들 수 있음

```
input_characters = number_of_sequences * sequence_length * characters_per_token
```

- **`input_characters`**: 토크나이저에 입력된 문자열에 있는 문자의 개수
- **`number_of_sequences`**: 토크나이저로부터 얻으려는 (잘린) 시퀀스의 개수(가령 100)
- **`sequence_length`**: 토크나이저가 반환한 각 시퀀스의 토큰 개수(가령 1,024)
- **`characters_per_tokens`**: 사전에 추정해야 하는 각 출력 토큰의 평균 문자 개수

- input_characters 개의 문자로 된 문자열을 입력하면 평균적으로 number_of_sequences개의 출력 시퀀스를 얻음
- number_of_sequences=100은 시퀀스를 100개 쌓고 너무 짧거나 긴 마지막 원소를 잃는다는 의미
  + 즉, 데이터셋의 1%의 손실이 발생
  

In [62]:
# 데이터 셋에 있는 토큰의 평균 문자 길이 예측

examples, total_characters, total_tokens = 500, 0, 0
dataset = load_dataset('transformersbook/codeparrot-train', split='train',
                       streaming=True)

for _, example in tqdm(zip(range(examples), iter(dataset)), total=examples):
    total_characters += len(example['content'])
    total_tokens += len(tokenizer(example['content']).tokens())

characters_per_token = total_characters / total_tokens

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2605 > 1024). Running this sequence through the model will result in indexing errors


In [63]:
print(characters_per_token)

3.6233025034779565


모델에게 일정한 길이의 입력을 주입하도록 사용자 정의 **IterableDataset**(파이토치에서 제공하는 헬퍼 클래스)를 만들기 위한 준비 완료

- IterableDataset을 상속해서 방금 살펴본 로직을 기반으로 아래 원소를 반환하는 __iter__() 함수 작성

In [64]:
import torch
from torch.utils.data import IterableDataset

class ConstantLengthDataset(IterableDataset):

    def __init__(self, tokenizer, dataset, seq_length=1024,
                 num_of_sequences=1024, chars_per_token=3.6):
        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.eos_token_id
        self.dataset = dataset
        self.seq_length = seq_length
        self.input_characters = seq_length * chars_per_token * num_of_sequences

    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.input_characters:
                    m=f"Buffer full: {buffer_len}>={self.input_characters:.0f}"
                    print(m)
                    break
                try:
                    m=f"Fill buffer: {buffer_len}<{self.input_characters:.0f}"
                    print(m)
                    buffer.append(next(iterator)["content"])
                    buffer_len += len(buffer[-1])
                except StopIteration:
                    iterator = iter(self.dataset)

            all_token_ids = []
            tokenized_inputs = self.tokenizer(buffer, truncation=False)
            for tokenized_input in tokenized_inputs['input_ids']:
                all_token_ids.extend(tokenized_input + [self.concat_token_id])

            for i in range(0, len(all_token_ids), self.seq_length):
                input_ids = all_token_ids[i : i + self.seq_length]
                if len(input_ids) == self.seq_length:
                    yield torch.tensor(input_ids)

__iter__() 메서드는 충분한 개수의 문자를 포함할 때까지 문자열 버퍼를 채움
- 버퍼 안의 모든 원소는 토큰화되어 EOS 토큰으로 연결되고 그 다음 all_token_ids에 있는 긴 시퀀스가 seq_length 크기로 나뉨
- 일반적으로 패딩된 가변 길이 시퀀스를 쌓을 때는 훈련 시 패딩을 무시하도록 어텐션 마스크가 필요함
- 여기서는 동일한 (최대) 길이 시퀀스만 제공하므로 마스크가 필요치 않으며 input_ids만 반환


In [65]:
shuffled_dataset = dataset.shuffle(buffer_size=100)
constant_length_dataset = ConstantLengthDataset(tokenizer, shuffled_dataset,
                                                num_of_sequences=10)
dataset_iterator = iter(constant_length_dataset)

lengths = [len(b) for _, b in zip(range(5), dataset_iterator)]
print(f"시퀀스 길이: {lengths}")

Fill buffer: 0<36864
Fill buffer: 3506<36864
Fill buffer: 12268<36864
Fill buffer: 13514<36864
Fill buffer: 35500<36864
Buffer full: 38137>=36864
시퀀스 길이: [1024, 1024, 1024, 1024, 1024]


의도 대로 작동해 모델에 전달할 일정 길이의 입력을 얻음
- ConstantLengthDataset를 만들기 전에 원시 데이터 셋을 섞음
- 반복 가능한 데이터 셋이므로 처음에 전체 데이터셋을 섞을 수 없음
- 대신 데이터셋에서 원소를 가져오기 전에 buffer_size 크기의 버퍼를 할당하고 버퍼 안의 원소를 섞음

### 10.3.4 훈련 루프 정의하기

사용자 정의 언어 모델을 훈련할 때 분명한 제약 조건은 GPU의 메모리 제한
- 최신 그래픽 카드에서도 GPT-2 규모의 모델을 적절한 시간 내에 훈련하지 못함

이 예제에서는 여러개의 GPU를 훈련에 사용하기 위한 **데이터 병렬화(data parallelism)**를 구현함
- 다행히 허깅페이스 엑셀러레이트를 사용해 예제 코드를 확장할 수 있음
- 허깅페이스 엑셀러레이트 라이브러리는 분산 훈련을 위해 하드웨어를 쉽게 바꿀 수 있도록 고안됨

허깅페이스 엑셀러레이트는 혼합 정밀도와 어떤 종류의 분산 환경(단일 GPU, 다중 GPU, TPU)에서도 훈련 스크립트를 실행할 수 있는 간편한 API를 제공함
- 동일한 코드를 디버깅하기 위해 로컬 컴퓨터에서 실행하거나 최종 훈련을위해 대규모 훈련 클러스터에서도 실행이 가능함
- 기본 파이토치 훈련 루프에서 약간만 변경하면 됨

```python
import torch
import torch.nn.functional as F
from datasets import load_dataset
from accelerate improt Accelerator

# device = 'cpu'

accelerator = Accelerator()
# model = torch.nn.Transformer().to(device)
model = torch.nn.Transformer()
optimizer = torch.optim.Adam(model.parameters())
dataset = load_dataset('my_dataset')
data = torch.utils.data.DataLoader(dataset, shuffle = True)
model, optimizer, data = accelerator.prepare(model, optimizer, data)

model.tarin()
for epoch in range(10):
  for source, targets in data:
#    source = source.to(device)
#    targets = targets.to(device)
    optimizer.zero_grad()
    output = model(source)
    loss = F.cross_entropy(output, targets)
#    loss.backward()
    accelerator.backward(loss)
    optimizer.step()
```

바뀐 부분 중 핵심을 prepare() 메서드 호출임

- 모델, 옵티마이저, 데이터로더를 모두 준비하고 인프라에 분산함
- 이렇게 파이토치 훈련 루프를 조금 바꿔 다양한 인프라로 훈련을 확장함

훈련 스크립트 생성
- 훈련을 위한 하이퍼파라미터 설정, 접근이 용이하도록 Namespace로 감쌈

In [66]:
from argparse import Namespace

# 작은 모델에 해당하는 파라미터
config = {"train_batch_size": 2, # 12
          "valid_batch_size": 2, # 12
          "weight_decay": 0.1,
          "shuffle_buffer": 1000,
          "learning_rate": 2e-4, # 5e-4
          "lr_scheduler_type": "cosine",
          "num_warmup_steps": 750, # 2000
          "gradient_accumulation_steps": 16, # 1
          "max_train_steps": 50000, # 150000
          "max_eval_steps": -1,
          "seq_length": 1024,
          "seed": 1,
          "save_checkpoint_steps": 50000} # 15000

args = Namespace(**config)

그 다음 훈련을 위한 로깅을 설정
- 모델의 밑바닥부터 훈련하기 위한 모든 정보를 저장하고 참조하는 프레임워크  
- 주어진 문제와 선호하는 바에 따라 로깅 프레임워크를 추가하거나 삭제할 수 있음
  + 표준 파이썬 Logger
  + 텐서보드
  + wandb(Weights & Biases)

In [68]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [69]:
from torch.utils.tensorboard import SummaryWriter
import logging
import wandb

def setup_logging(project_name):
    logger = logging.getLogger(__name__)
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO, handlers=[
        logging.FileHandler(f"log/debug_{accelerator.process_index}.log"),
        logging.StreamHandler()])
    if accelerator.is_main_process: # 로깅을 한 번만 설정합니다.
        wandb.init(project=project_name, config=args)
        run_name = wandb.run.name
        tb_writer = SummaryWriter()
        tb_writer.add_hparams(vars(args), {'0': 0})
        logger.setLevel(logging.INFO)
        datasets.utils.logging.set_verbosity_debug()
        transformers.utils.logging.set_verbosity_info()
    else:
        tb_writer = None
        run_name = ''
        logger.setLevel(logging.ERROR)
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()
    return logger, tb_writer, run_name

각 워커(worker)는 고유한 accelerator.process_index를 받음
- 이를 이용해 각 워커의 로그를 개별 파일에 기록함

메인 워커에서만 true인 accelerator.is_main_process 속성을 사용
- 이를 사용해 텐서보드와 wandb의 로거가 여러번 초기화되지 않게 다른 워커에서 로깅 수준을 낮춤

나중에 허브에서 실험 결과에 이름을 부여하기 위해 자동으로 생성된 고유한 wandb.run.name을 반환함


In [70]:
def log_metrics(step, metrics):
    logger.info(f"Step {step}: {metrics}")
    if accelerator.is_main_process:
        wandb.log(metrics)
        [tb_writer.add_scalar(k, v, step) for k, v in metrics.items()]

텐서보드와 wandb에 측정값을 기록할 함수도 정의
- 여기서도 accelerator.is_main_process를 다시 사용해 측정값이 워커마다 저장되지 않고 한 번만 기록되게 함

그 다음 사용자 정의 ConstantLengthDataset 클래스로 훈련 세트와 검증 세트를 위한 데이터 로더를 만드는 함수를 작성

In [89]:
!pip install accelerator

In [71]:
from torch.utils.data.dataloader import DataLoader

def create_dataloaders(dataset_name):
    train_data = load_dataset(dataset_name+'-train', split="train",
                              streaming=True)
    train_data = train_data.shuffle(buffer_size=args.shuffle_buffer,
                                    seed=args.seed)
    valid_data = load_dataset(dataset_name+'-valid', split="validation",
                              streaming=True)

    train_dataset = ConstantLengthDataset(tokenizer, train_data,
                                          seq_length=args.seq_length)
    valid_dataset = ConstantLengthDataset(tokenizer, valid_data,
                                          seq_length=args.seq_length)

    train_dataloader=DataLoader(train_dataset, batch_size=args.train_batch_size)
    eval_dataloader=DataLoader(valid_dataset, batch_size=args.valid_batch_size)
    return train_dataloader, eval_dataloader

마지막에 배치처리를 위해 데이터셋을 DataLoader로 감쌈
- 허깅페이스 엑셀러레이트가 배치를 각 워커로 분산


최적화 하이퍼파라미터 구현
- 메인 루프에서 옵티마이저와 학습률을 설정하겠지만, 가중치 감쇠를 받아야 하는 파라미터를 구분하기 위해 헬퍼 함수를 정의
- 보통 절편과 LayerNorm의 가중치에는 가중치 감쇠를 적용하지 않음


In [72]:
def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [{'params': params_with_wd, 'weight_decay': args.weight_decay},
            {'params': params_without_wd, 'weight_decay': 0.0}]

평가 세트에서 손실과 복잡도(perplexity)를 계산하는 평가 함수를 추가

In [82]:
def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch, labels=batch)
        loss = outputs.loss.repeat(args.valid_batch_size)
        losses.append(accelerator.gather(loss))
        if args.max_eval_steps > 0 and step >= args.max_eval_steps:
          break
    loss = torch.mean(torch.cat(losses))
    try:
		    perplexity = torch.exp(loss)
    except OverflowError:
		    perplexity = torch.tensor(float("inf"))
    return loss.item(), perplexity.item()

복잡도는 모델의 출력 확률 분포가 얼마나 타깃 토큰을 잘 예측하는지 측정함
- 복잡도가 낮을수록 성능이 좋음
- 복잡도는 모델 출력에서 얻은 크로스 엔트로피 손실에 지수 함수를 적용해 계산
- 특히 훈련 초기에 손실이 높을 때 복잡도을 계산하면 수치적으로 오버 플로가 발생할 수 있음
  + 이 경우 오류를 캐치해 이 경우 복잡도를 무한대로 설정
  

In [92]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.2 MB/s eta 0:00:00


In [96]:
!mkdir log

In [97]:
!touch log/debug_0.log

훈련 스크립트의 핵심 부분 작성(실제로는 wandb 로그인이 필요)

1. 모델 저장

2. 최적화

3. 평가

4. 그레디언트 누적과 체크 포인팅


훈련 스크립트를 codeparrot_training.py 파일에 저장

```python

from accelerate import Accelerator

# project_name = "huggingface/pytorch-transformers-examples"
# dataset_name = "wikitext"

set_seed(args.seed)

# 엑셀러레이트
accelerator = Accelerator()
samples_per_step = accelerator.state.num_processes * args.train_batch_size

# 로깅
logger, tb_writer, run_name = setup_logging(project_name.split("/")[1])
logger.info(accelerator.state)

# 모델과 토크나이저를 로드합니다
if accelerator.is_main_process:
    hf_repo = Repository("./", clone_from=project_name, revision=run_name)
model = AutoModelForCausalLM.from_pretrained("./", gradient_checkpointing=True)
tokenizer = AutoTokenizer.from_pretrained("./")

# 데이터셋과 데이터로더를 로드합니다
train_dataloader, eval_dataloader = create_dataloaders(dataset_name)

# 옵티마이저와 학습률 스케줄러를 준비합니다
optimizer = AdamW(get_grouped_params(model), lr=args.learning_rate)
lr_scheduler = get_scheduler(name=args.lr_scheduler_type, optimizer=optimizer,
                             num_warmup_steps=args.num_warmup_steps,
                             num_training_steps=args.max_train_steps,)
def get_lr():
    return optimizer.param_groups[0]['lr']

# `accelerator`로 모든 것을 준비합니다(매개변수 순서는 중요하지 않습니다)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader)

# 모델을 훈련합니다
model.train()
completed_steps = 0
for step, batch in enumerate(train_dataloader, start=1):
    loss = model(batch, labels=batch).loss
    log_metrics(step, {'lr': get_lr(), 'samples': step*samples_per_step,
                       'steps': completed_steps, 'loss/train': loss.item()})
    loss = loss / args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % args.gradient_accumulation_steps == 0:
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        completed_steps += 1
    if step % args.save_checkpoint_steps == 0:
        logger.info('Evaluating and saving model checkpoint')
        eval_loss, perplexity = evaluate()
        log_metrics(step, {'loss/eval': eval_loss, 'perplexity': perplexity})
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        if accelerator.is_main_process:
            unwrapped_model.save_pretrained("./")
            hf_repo.push_to_hub(commit_message=f'step {step}')
        model.train()
    if completed_steps >= args.max_train_steps:
        break

# 마지막 체크포인트를 평가하고 저장합니다
logger.info('Evaluating and saving model after training')
eval_loss, perplexity = evaluate()
log_metrics(step, {'loss/eval': eval_loss, 'perplexity': perplexity})
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
if accelerator.is_main_process:
    unwrapped_model.save_pretrained("./")
    hf_repo.push_to_hub(commit_message=f'final model')
```

분산 환경에서 모델을 훈련하는 방법은 모델 크기와 데이터 용량에 따라 다름
- 허깅페이스 엑셀러레이트에서 사요하는 방법은 **DataDistributedParallelism(DDP)** 이며, 단일 GPU에 맞는 것보다 더 큰 배치에서 모델을 빠르게 훈련한다는 강점이 있음

<img alt="DDP" caption="Illustration of the processing steps in DDP with four GPUs" src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter10_ddp.png?raw=1" id="ddp"/>

**[그림 10-6]** 4개의 GPU를 사용한 DDP

- 각 워커는 하나의 GPU로 구성. 허깅페이스 엑셀러레이트에서는 메인 프로세스에서 실행하는 데이터로더가 데이터 배치를 준비하고 모든 워커에 배치를 전달함
- 각 GPU는 데이터 배치를 받고 모델의 로컬 복사본을 사용해 정방향 패스와 역방향 패스로부터 손실과 누적 그래디언트를 계산
- 각 노드의 그레디언트는 올-리듀스(all-reduce) 패턴으로 평균되고 평균된 그레디언트는 각각의 워커로 다시 전달
- 각 노드에서 개별적으로 옵티마이저를 사용해 그레디언트가 적용
- 모든 모델이 업데이트되면 메인 워커가 새로운 배치를 준비하고 전체 과정을 다시 시작


### 10.3.5 훈련 실행

훈련 스크립트를 codeparrot_training.py 파일에 저장

아래 쉘 명령어를 통해 학습 실행
```
!pip install wandb
!git clone https://huggingface.co/transformersbook/codeparrot
!cd codeparrot
!pip install -r requirements.txt
!wandb login
!accelerate config
!accelerate launch codeparrot_training.py
```

전체 훈련 실행 후 실험 브랜치는 메인 브랜치로 머지해 허브로 푸쉬함
```
!git checkout main
!git merge <RUN_NAME>
!git push
```

## 10.4 결과 및 분석

파이프라인으로 작은 모델을 감싸고 이어서 샘플 코드 입력을 전달

In [99]:
from transformers import pipeline, set_seed

model_ckpt = 'transformersbook/codeparrot-small'
generation = pipeline('text-generation', model=model_ckpt, device=0)

config.json:   0%|          | 0.00/865 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/457M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/497k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/277k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

주어진 프롬프트에서 완성된 코드 후보를 생성하기 위해 생성 파이프라인을 사용
- 기본적으로 이 파이프라인은 사전에 정의된 길이에 도달할 때까지 코드를 생성
- 출력은 여러 개의 함수나 클래스를 포함할 수 있음
  + 따라서 출력을 간략하게 유지하기 위해 정규식으로 첫째로 등장한 함수나 클래스를 추출하는 first_block() 함수를 구현
  + 그 아래 complete_code() 함수는 이 로직을 적용하고 CodeParrot이 생성한 코드 완성을 출력

In [100]:
import re
from transformers import set_seed

def first_block(string):
    return re.split('\nclass|\ndef|\n#|\n@|\nprint|\nif', string)[0].rstrip()

def complete_code(pipe, prompt, max_length=64, num_completions=4, seed=1):
    set_seed(seed)
    gen_kwargs = {"temperature":0.4, "top_p":0.95, "top_k":0, "num_beams":1,
                  "do_sample":True,}
    code_gens = generation(prompt, num_return_sequences=num_completions,
                            max_length=max_length, **gen_kwargs)
    code_strings = []
    for code_gen in code_gens:
        generated_code = first_block(code_gen['generated_text'][len(prompt):])
        code_strings.append(generated_code)
    print(('\n'+'='*80 + '\n').join(code_strings))

예제 1) 사각형 면적 구하는 코드

In [101]:
prompt = '''def area_of_rectangle(a: float, b: float):
    """Return the area of the rectangle."""'''
complete_code(generation, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    return math.sqrt(a * b)

    return a * b / 2.0

    return a * b

    return a * b / a


예제 2) HTML에서 URL 추출

In [102]:
prompt = '''def get_urls_from_html(html):
    """Get all embedded URLs in a HTML string."""'''
complete_code(generation, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    if not html:
        return []
    return [url for url in re.findall(r'<a href="(/[^/]+/[^"]+?)">', html)]

    return [url for url in re.findall(r'<a href="(.*?)"', html)
            if url]

    return [url for url in re.findall(r'<a href="(/.*)",', html)]

    return re.findall(r'<a href="(.*?)" class="url"[^>]*>', html)


예제 3) 허깅페이스 홈페이지로 함수 테스트

In [103]:
import requests

def get_urls_from_html(html):
    return [url for url in re.findall(r'<a href="(.*?)"', html) if url]

print(" | ".join(get_urls_from_html(requests.get('https://hf.co/').text)))

/models | /spaces/InstantX/InstantID | /spaces/FaceOnLive/ID-Document-Recognition-SDK | /spaces/vikhyatk/moondream1 | /spaces/lmsys/chatbot-arena-leaderboard | /spaces/Recognito/FaceAnalysis | /spaces | /datasets | /docs/transformers | /docs/diffusers | /docs/safetensors | /docs/huggingface_hub | /docs/tokenizers | /docs/peft | /docs/transformers.js | /docs/timm


> **NOTE**: 다음 코드 블록에서 대용량 GPT-2 체크포인트를 메모리에 로드합니다. 코랩이나 캐글 같은 플랫폼에서는 램이나 GPU 메모리가 부족하기 때문에 인스턴스가 종료될 수 있습니다. 이런 경우 `model_ckpt = "transformersbook/codeparrot-small"`로 바꾸어 작은 체크포인트를 사용하면 이 예제를 실행할 수 있습니다.

예제 4) numpy 활용 방식 생성

In [104]:
model_ckpt = 'transformersbook/codeparrot'
generation = pipeline('text-generation', model=model_ckpt, device=0)

prompt = '''# a function in native python:
def mean(a):
    return sum(a)/len(a)

# the same function using numpy:
import numpy as np
def mean(a):'''
complete_code(generation, prompt, max_length=64)

config.json:   0%|          | 0.00/959 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/497k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/277k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



    return np.mean(a)

    return sum(a)/len(a)

    return np.mean(a)

    return sum(a)/len(a)


예제 5) 사이킷런 모델 활용 예제 생성

In [105]:
prompt = '''X = np.random.randn(100, 100)
y = np.random.randint(0, 1, 100)

# fit random forest classifier with 20 estimators'''
complete_code(generation, prompt, max_length=96)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



reg = DummyRegressor()

forest = RandomForestClassifier(n_estimators=20)

forest.fit(X, y)

clf = ExtraTreesClassifier(n_estimators=100, max_features='sqrt')
clf.fit(X, y)

clf = RandomForestClassifier(n_estimators=20, n_jobs=n_jobs, random_state=1)
clf.fit(X, y)

clf = RandomForestClassifier(n_estimators=20)
clf.fit(X, y)


생성된 텍스트의 품질을 측정하는 방법 중 하나로 많이 사용되는 BLEU 지표는 일반적인 한계가 있지만, 이 문제에는 특히 잘 맞지 않음
  + BLEU 점수는 참조 텍스트와 생성 텍스트 간의 n-gram 중복을 측정하기 때문

소프트웨어 개발 분야에서는 유닛 테스트와 같이 코드 품질을 측정할 때 신뢰할 만한 좋은 방법이 있으며 이런 방법을 통해 OpenAI Codex 모델을 평가함
- 코딩 작업을 위해 생성한 여러 코드를 일련의 단위 테스트를 통해 실행하고 테스트를 통과한 비율을 계산함
- CodeParrot이 HumanEval 벤치마크에서 어떻게 동작하는지 다룬 포스팅(https://huggingface.co/blog/codeparrot) 참조



## 10.5 결론

**수행 과정 요약**

1. 대규모 언어 모델을 사전 훈련하기 위해 대규모 데이터셋 직접 구축

2. 구축한 데이터셋으로 파이썬 코드를 효율적으로 인코딩하는 사용자 정의 토크나이저 생성

3. 허깅페이스 엑셀러레이트를 사용해 모든 것을 연결

4. 다중 GPU 인프라 또는 단일 인프라에서 소규모와 대규모 GPT-2 모델 훈련

5. 모델 출력을 확인하면서 모델이 적절하게 이어지는 코드를 생성할 수 있는지 확인